# 1. Connect to IB 

In [ ]:

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

# 2. Market Segment Analysis

In [ ]:
import market_analyzer as ma

# Initialize analyzer
analyzer = ma.IBMarketAnalyzer(ib)

# IMPORTANT: Set up the sectors first
analyzer.set_sectors()  # This was missing in the previous code

# Now get the full analysis
market_df, sector_df = analyzer.create_analysis_report()

# Then filter the sectors in various ways:

# Example 1: Get top 3 sectors with positive momentum and increasing volume
strong_sectors = analyzer.filter_sectors(
    sector_df,
    min_momentum=0,
    volume_trend='Increasing',
    top_n=3,
    sort_by='Overall Score'
)

# Example 2: Get all sectors outperforming SPY with minimum 2% return
outperformers = analyzer.filter_sectors(
    sector_df,
    outperforming_spy=True,
    min_return=2.0,
    sort_by='Return (%)'
)

# Example 3: Get sectors with strong relative strength and positive momentum
strong_rs = analyzer.filter_sectors(
    sector_df,
    min_rs=1.0,
    min_momentum=0,
    sort_by='Relative Strength'
)

# Example 4: Custom combination of filters
custom_filter = analyzer.filter_sectors(
    sector_df,
    min_return=0,
    min_rs=0,
    volume_trend='Increasing',
    sort_by='Overall Score',
    top_n=5
)

ib.disconnect()

print("=== Market Analysis ===")
display(market_df)

print("\n=== Sector Analysis ===")
display(sector_df)


try:
    # Print results
    print("\n=== Strong Sectors ===")
    display(strong_sectors)

    print("\n=== Outperformers ===")
    display(outperformers)

    print("\n=== Strong Relative Strength ===")
    display(strong_rs)

    print("\n=== Custom Filter ===")
    display(custom_filter)

except Exception as e:
    print(f"An error occurred: {str(e)}")

# 3. Scanner

In [ ]:
import scanner.scanner as scanner

stockbot = scanner.StockbotScanner(ib)
stockbot.multiscan(scan_code='TOP_PERC_GAIN', price=(1, 100), volume=100_000, change_perc=4, market_cap=100)
stockbot.scan_results_df

# 4. Fundamentals

In [ ]:
import fundamentals

from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

# # Connect to IB
# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)

try:
    stock_info = fundamentals.get_stock_info(ib, 'AAPL', current_volume=1000000)
    print(f"Current Price: ${stock_info.current_price:.2f}")
    print(f"52-week Range: ${stock_info.low_52week:.2f} - ${stock_info.high_52week:.2f}")
    print(f"Volume vs 10-day avg: {stock_info.volume_vs_10day_avg_pct:.2f}%")
except Exception as e:
    print(f"Error getting stock info: {e}")
finally:
    ib.disconnect()

stock_info.__dict__


In [ ]:
import my_ib_utils
import fundamentals

# Create rate limiter
rate_limiter = my_ib_utils.IBRateLimiter(ib, requests_per_second=2)

# Create Fundamentals instance
fund = fundamentals.Fundamentals(ib, rate_limiter)

# Get fundamentals for multiple stocks
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
data = fund.get_fundamentals(symbols)

# Convert to DataFrame and filter
df = fund.to_dataframe()
filtered_df = fund.filter_stocks({
    'pe_ratio': (0, 30),
    'market_cap': (1e11, None)
})

filtered_df

# 5. Daily Stock Data

In [10]:
import os

# Go up one level from scanner folder, then into data folder
current_dir = os.getcwd()  # Gets current directory (scanner)
parent_dir = os.path.dirname(current_dir)  # Goes up one level to stockbot4
data_folder = os.path.join(parent_dir, 'data')  # Goes into data folder
os.chdir(data_folder)

import historical_data as hd

symbol = 'ROKU'
interval = '1 day'
data = hd.get_hist_data(symbol, '52 weeksAgo', 'now', interval)
display(data)

Loading data from historical_data/ROKU_1_day.csv
Stored data: 478 rows of data
Loading data from historical_data/ROKU_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-11-21,94.00,94.99,91.96,93.40,36580
2023-11-22,93.57,95.61,92.55,95.00,31084
2023-11-24,94.80,96.00,93.87,94.93,17928
2023-11-27,96.03,103.57,95.23,103.30,138637
2023-11-28,103.47,107.16,102.23,107.05,99003
...,...,...,...,...,...
2024-11-18,71.10,74.44,69.89,74.00,34404
2024-11-19,74.09,75.02,72.18,73.75,21912
2024-11-20,73.06,73.11,65.97,68.70,69952


# 6. Daily TA

# 7. Filter TA

# 8. Daily Scores

# 9. Final Watch List